In [37]:
import numpy as np
import pandas as pd
import os
import time
import scanpy as sc
from dask import dataframe as dd
import dask
import pickle
import numpy
import h5py
import tables

In [3]:
control_count_matrix_filename = './Tran_RGC_scRNA/unzipped/GSE137398_ONCRGCs_control_count_mat.csv'

In [8]:
import gc
gc.collect()

22

### Read large csv with pandas no chunks

In [32]:
start = time.time()
pandas_df = pd.read_csv(control_count_matrix_filename)
end = time.time()
print("Read csv with pandas without chunks: ",(end-start),"sec")

Read csv with pandas without chunks:  171.90068125724792 sec


In [59]:
pandas_df_to_np = pandas_df.to_numpy()

### Read large csv with npy

In [56]:
np.save("npy.npy", pandas_df_to_np)

In [9]:
start = time.time()
np_df = np.load("npy.npy", allow_pickle=True)
print(f"Read with npy: {time.time()-start} sec")

Read with npy: 15.772084951400757 sec


### Read large csv with pickle

In [39]:
with open("test.pkl", "wb") as f:
    pickle.dump(pandas_df_to_np, f)

In [41]:
start = time.time()
with open("test.pkl", "rb") as f:
        pickle_df = pickle.load(f)
print(f"Read with pickle: {time.time()-start} sec")       

Read with pickle: 16.956642389297485 sec


### Read large csv with h5

In [ ]:
f = h5py.File("hdf5.h5", "w")
f.create_dataset("data", data=pandas_df_to_np)
f.close()

In [ ]:
start = time.time()
f = h5py.File("out.h5", "r")
out = f["data"][()]
f.close()
print(f"Read with h5: {time.time()-start} sec")    

### Read large csv with scanpy

In [27]:
start = time.time()
scanpy_df = sc.read_csv(control_count_matrix_filename, first_column_names=True)
print(f"Read csv with scanpy: {time.time()- start} sec")

Read csv with scanpy: 56.38237380981445 sec


In [55]:
scanpy_df.shape

(40790, 17887)

### Read large csv with pandas chunks

In [31]:
start = time.time()
pandas_chunk_df = pd.read_csv(control_count_matrix_filename,chunksize=1000000)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")
pd_concat_df = pd.concat(pandas_chunk_df)
print(f"Concat with pandas chunk {time.time()-start} sec")

Read csv with chunks:  0.007316112518310547 sec
Concat with pandas chunk 166.26809334754944 sec


### Read large csv with dask

In [57]:
start = time.time()
dask_df = dd.read_csv(control_count_matrix_filename, sample=1000000)
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  58.971813440322876 sec


In [58]:
start = time.time()
dask_df.head()
print(f"Head of dask dataframe {time.time()-start} sec")

Head of dask dataframe 148.48653936386108 sec


In [ ]:
dask_df_compute = dask_df.mean()

In [ ]:
np.asarray(dask_df)